<a href="https://colab.research.google.com/github/AndrewstheBuilder/ScratchNeuralNetworks/blob/main/Iris_Micrograd_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Load the Iris Dataset
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels (0, 1, or 2)


# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [16]:
# Do preprocessing
print('Features of dataset:',X[0])
print('Labels of dataset:',y)
X.shape

Features of dataset: [-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
Labels of dataset: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


(150, 4)

In [1]:
pip install micrograd_andrews

In [18]:
import math

def softmax(logits):
    exps = [logit.exp() for logit in logits]
    sum_exps = sum(exps)
    return [exp_i / sum_exps for exp_i in exps]

def cross_entropy(probs, label):
    # Convert label to one-hot encoding
    one_hot = [0] * len(probs)
    one_hot[label] = 1
    # Compute cross-entropy loss
    loss = -sum((p_i + 1e-9).log() * y_i for p_i, y_i in zip(probs, one_hot))
    return loss

In [27]:
# Use Micrograd MLP to solve the Iris dataset.
from micrograd_andrews.engine import Value
from micrograd_andrews.nn import Neuron, Layer, MLP

model = MLP(X.shape[0], [2,3])
# Begin gradient descent iterations

# Hyperparameters
learning_rate = 0.05
epochs = 50
total_loss = 0.0
for epoch in range(epochs):
  correct = 0
  for i in range(len(X_train)):
    logits = model(X_train[i])
    # print(logits)
    probs = softmax(logits)
    y_true = y_train[i]
    loss = cross_entropy(probs, y_true)
    total_loss += loss.data

    # Prediction
    pred_label = probs.index(max(probs, key=lambda p: p.data))
    if pred_label == y_true:
      correct += 1

    # Back propagation
    model.zero_grad()
    loss.backward()

    # Update parameters
    for p in model.parameters():
      p.data -= learning_rate * p.grad
      avg_loss = total_loss / len(X_train)

  accuracy = correct / len(X_train) * 100
  print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/50, Loss: 0.7606, Accuracy: 78.33%
Epoch 2/50, Loss: 1.2923, Accuracy: 85.83%
Epoch 3/50, Loss: 1.6955, Accuracy: 88.33%
Epoch 4/50, Loss: 2.0011, Accuracy: 93.33%
Epoch 5/50, Loss: 2.2333, Accuracy: 95.83%
Epoch 6/50, Loss: 2.4165, Accuracy: 95.83%
Epoch 7/50, Loss: 2.5683, Accuracy: 96.67%
Epoch 8/50, Loss: 2.6998, Accuracy: 96.67%
Epoch 9/50, Loss: 2.8175, Accuracy: 96.67%
Epoch 10/50, Loss: 2.9254, Accuracy: 96.67%
Epoch 11/50, Loss: 3.0260, Accuracy: 96.67%
Epoch 12/50, Loss: 3.1211, Accuracy: 96.67%
Epoch 13/50, Loss: 3.2118, Accuracy: 96.67%
Epoch 14/50, Loss: 3.2991, Accuracy: 96.67%
Epoch 15/50, Loss: 3.3835, Accuracy: 96.67%
Epoch 16/50, Loss: 3.4656, Accuracy: 96.67%
Epoch 17/50, Loss: 3.5457, Accuracy: 96.67%
Epoch 18/50, Loss: 3.6240, Accuracy: 96.67%
Epoch 19/50, Loss: 3.7009, Accuracy: 95.83%
Epoch 20/50, Loss: 3.7766, Accuracy: 95.83%
Epoch 21/50, Loss: 3.8512, Accuracy: 95.83%
Epoch 22/50, Loss: 3.9249, Accuracy: 95.83%
Epoch 23/50, Loss: 3.9979, Accuracy: 95.8

In [28]:
correct = 0

for i in range(len(X_test)):
    y_true = y_test[i]

    # Forward pass
    logits = model(X_test[i])
    probs = softmax(logits)

    # Prediction
    pred_label = probs.index(max(probs, key=lambda p: p.data))
    if pred_label == y_true:
        correct += 1

test_accuracy = correct / len(X_test) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 96.67%
